In [59]:
from toolbar.TN_WaveActivityFlux import TN_WAF_3D
from toolbar.curved_quivers.modplot import Curlyquiver
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from toolbar.masked import masked   # 气象工具函数
from toolbar.pre_whitening import ws2001
import pandas as pd
import tqdm

#以下为数据读取部分，最终所得z300，u300，v300为2014年11月的300hPa位势高度场，UV风场的月平均；
#z_tmp，u_tmp，v_tmp为1979-2018年11月的气候态；
#所有数据来自ECMWF的ERA-Interim数据集
data_year = ['1961', '2022']
CN051_1 = xr.open_dataset(r"C:\Users\10574\Desktop\CN05.1_Tmax_1961_2021_daily_025x025.nc")
CN051_2 = xr.open_dataset(r"C:\Users\10574\Desktop\CN05.1_Tmax_2022_daily_025x025.nc")
Tmax_cn051 = xr.concat([CN051_1, CN051_2], dim='time').sel(time=slice(data_year[0]+'-01-01', data_year[1]+'-12-31'))
Tmax_cn051 = Tmax_cn051.sel(time=Tmax_cn051['time.month'].isin([6, 7, 8]))
lon = Tmax_cn051['lon']
lat = Tmax_cn051['lat']

time_series = np.zeros((len(Tmax_cn051['time'].groupby('time.year')), 92))
station_num = masked((Tmax_cn051-Tmax_cn051+1).sel(time='2022-07-01'), r"D:\PyFile\map\地图边界数据\长江区1：25万界线数据集（2002年）\长江区.shp")['tmax']  # 掩膜处理得长江流域站点数
station_num = station_num.sum()  # 长江流域格点数

C:\Program Files (x86)\Anaconda3\lib\site-packages\xarray\coding\times.py:170: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


3802.0

In [116]:
YR_Tmax = masked(Tmax_cn051, r"D:\PyFile\map\地图边界数据\长江区1：25万界线数据集（2002年）\长江区.shp")
for iyear in tqdm.trange(1961, 1961+len(time_series)):
    time_continue = np.zeros((YR_Tmax['tmax'].shape[1], YR_Tmax['tmax'].shape[2]))  #   时间持续系数
    # 判断闰年
    if iyear % 400 == 0:
        leap_year = 1
    elif iyear % 4 == 0 and iyear % 100!= 0:
        leap_year = 1
    else:
        leap_year = 0
    for iday in range(92):
        iYR_Tmax = YR_Tmax.sel(time=YR_Tmax['time.year'].isin([iyear]))
        iYR_Tmax = iYR_Tmax.sel(time=iYR_Tmax['time.dayofyear'].isin([iday+leap_year+152]))
        iYR_Tmax = iYR_Tmax - 34.  # 减去34℃(阈值为35)
        time_continue = np.where(np.isnan(time_continue), 0, time_continue)  #   时间持续系数初始化
        time_continue += np.where(iYR_Tmax['tmax'] >= 1., 1, np.nan)[0]   #   时间持续系数
        S_index = np.where(iYR_Tmax['tmax'] >= 1., 1, 0).sum() / station_num  #  面积覆盖系数
        iYR_Tmax = iYR_Tmax.where(iYR_Tmax >= 1., np.nan)  #   高温强度系数  # 大于35℃的置为1，小于35℃的置为nan
        time_series[iyear-1961, iday] = S_index * np.nansum((iYR_Tmax + time_continue)['tmax']) #   面积覆盖系数 * Σ(时间持续系数*面积覆盖系数)

100%|██████████| 62/62 [01:03<00:00,  1.03s/it]


In [146]:
ols = np.load(r"D:\PyFile\paper1\OLS35_detrended.npy")
np.corrcoef(time_series[18:].mean(axis=1), ols)

array([[1.        , 0.58617444],
       [0.58617444, 1.        ]])

In [ ]:
def plot_waf(location, z):
    sub_ax = fig.add_subplot(location, projection=ccrs.PlateCarree(central_longitude=180))
    waf_x, waf_y = TN_WAF_3D(z_clim, u_clim, v_clim, z - z_clim, filt=1, filtmode='mix')
    # 生成20个等高线
    level = np.linspace(-np.nanmax(np.abs((z - z_clim).sel(level=200))),
                        np.nanmax(np.abs((z - z_clim).sel(level=200))), 21)
    sub_ax.contour(lon, lat, (z - z_clim).sel(level=200), levels=level, transform=ccrs.PlateCarree(central_longitude=0), colors='white', linewidth=0.3)
    sub_ax.contourf(lon, lat, (z - z_clim).sel(level=200), levels=level, transform=ccrs.PlateCarree(central_longitude=0), cmap='coolwarm')
    WAF = Curlyquiver(sub_ax, lon, lat, waf_x, waf_y, regrid=30, lon_trunc=0, arrowsize=1.3, scale=100, linewidth=1.3,
                  color='black', transform=ccrs.PlateCarree(central_longitude=0), mode='loose')
    WAF.key(fig, U=.5, label='0.5 m$^2$/s$^2$', lr=1)
    sub_ax.coastlines()
#以下为绘图部分
fig = plt.figure(figsize=(10, 10), dpi=600)
# 画出2000年夏季6-8月平均的WAF
z300 = era5['z'].sel(date=era5['date.year'].isin([2000])).mean(dim='date')
plot_waf(111, z300)
plt.show()